In [1]:
import torch
from torch import nn
from entmax import entmax_bisect
from torch.autograd import grad

In [4]:
alpha = nn.Parameter(torch.Tensor([1.5]))

In [5]:
alpha

Parameter containing:
tensor([1.5000], requires_grad=True)

In [2]:
class AlphaChooser(torch.nn.Module):

    def __init__(self, head_count):
        """head_count (int): number of attention heads"""
        super(AlphaChooser, self).__init__()
        self.pre_alpha = nn.Parameter(torch.randn(head_count))

    def forward(self):
        alpha = 1 + torch.sigmoid(self.pre_alpha)
        return torch.clamp(alpha, min=1.01, max=2)

In [3]:
att_scores = torch.rand(128, 12, 36, 1024)

In [5]:
alpha = AlphaChooser(12)

In [6]:
alpha()

tensor([1.0335, 1.5665, 1.4902, 1.4036, 1.7816, 1.3268, 1.2720, 1.4511, 1.2789,
        1.8413, 1.4065, 1.8338], grad_fn=<ClampBackward>)

In [11]:
p = entmax_bisect(att_scores,alpha()[0])

In [23]:
p.mean()

tensor(0.0010, grad_fn=<MeanBackward0>)

In [22]:
p.mean().backward()